In [5]:
import math


class Value:
    def __init__(self, data, _parents = (), _op = ''):
        self.data = data
        self._prev = set(_parents)
        self.grad = 0
        self._op = _op
        self.__update_parent_grad__ = lambda:None
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
     
    def backprop(self):
        self.__update_parent_grad__()
        for node in self._prev:
            node.backprop()
            
    def __add__(self, other):
        if type(other) is not Value:
            other = Value(other)
        
        output = Value(self.data + other.data, (self, other), '+')
        
        def __update_parent_grad__():
            self.grad += output.grad
            other.grad += output.grad
            
        output.__update_parent_grad__ = __update_parent_grad__
        return output
        
    def __radd__(self, other):
        return self + other
    
    def __mul__(self, other):
        if type(other) is not Value:
            other = Value(other)
        output = Value(self.data * other.data, (self, other), '*')
        
        def __update_parent_grad__():
            self.grad += other.data * output.grad
            other.grad += self.data * output.grad
        output.__update_parent_grad__ = __update_parent_grad__
        return  output
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        if type(other) is not Value:
            return self/Value(other)
        return self * other ** -1 
    
    def __rtruediv__(self, other):
        if type(other) is not Value:
            return Value(other)/self
        return other.__truediv__(self)
    
    def __pow__(self, power, modulo=None):
        if type(power) is not Value:
            return self ** Value(power)
        
        output = Value(self.data**power.data, (self, power), '**')
        def __update_parent_grad__():
            self.grad += (power.data * self.data ** (power.data-1))*output.grad
            power.grad += ((self.data ** power.data) * math.log(self.data))*output.grad
        self.__update_parent_grad__ = __update_parent_grad__
        return output
        

In [9]:
a = Value(1)
b = Value(2)
h = 1e-6
a_h = a + h
c = a+b
c_h = a_h+b
d = c*a

d_h = c_h*a_h

In [60]:
d.grad = 1
d.backprop()

In [61]:
c.grad

1

In [62]:
a.grad

4

In [63]:
b.grad

1

In [65]:
(d_h.data-d.data)/h

4.000000999759834

In [58]:
d

Value(data=3)

In [63]:
a = Value(5)
c = a*2
b = a*3

result = (b + c)/a
result.grad = 1
result.backprop()
result

Value(data=5.0)

In [62]:
c.grad

0.2

In [6]:
a = Value(3)
b = Value(2)
c = 1/b
c.grad = 1
c.backprop()

In [58]:
b.grad

tensor([0.2000], dtype=torch.float64)

In [16]:
b.grad

-0.25

In [17]:
import torch

In [53]:
a = torch.Tensor([5]).double()
a.requires_grad = True

b = a*3
c = a*6
b.retain_grad()
c.retain_grad()
result = (c+b)/a


In [55]:
result.backward()

In [46]:
result

tensor([9.], dtype=torch.float64, grad_fn=<DivBackward0>)

In [57]:
c.grad

tensor([0.2000], dtype=torch.float64)